### In this notebook we are going to create a classifier that respects a fairness metric. Our choice is demographic parity. 
### In order to achieve this, we are going to take a normal classificator and post process the predictions.
### The HopefullyFairClassifier has the same fit and predict methods as a normal classifier, but it also takes information regarding the sensitive attribute. 
### We are starting with some assumptions: we have one binary sensitive attribute and binary output.
### In the fit method, we are using a large subset of the training data in order to train the naive classifier, and a smaller subset in order to find two tresholds, one for the priviledged and the other for the unpriviledged group. The main idea of our method is to have two different treshodls, one for the priviledged and the other for the unpriviledged group. We are going to learn, on a validation set, the tresholds that together satisfy the best the demographic parity. 
### When predicting, depending on the sensitive attribute, we are going to use the appropriate treshold. 

In [62]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [63]:
# Metrics for demographic parity


def dp_diff(y_priv, y_unpriv):
    
    dp_priv = sum(y_priv)/len(y_priv)
    dp_unpriv = sum(y_unpriv)/len(y_unpriv)
    return dp_priv-dp_unpriv


def disparate_impact(y_priv, y_unpriv):
    dp_priv = sum(y_priv)/len(y_priv)
    dp_unpriv = sum(y_unpriv)/len(y_unpriv)
    return dp_unpriv/dp_priv

In [64]:
class HopefullyFairClassifier:
    def __init__(self, priv, unpriv, attribute) -> None:
        self.naive_classifier = LogisticRegression(penalty="l2", solver='lbfgs', C=0.01)
        self.treshold_priv = None
        self.treshold_unpriv = None 
        self.priv = priv 
        self.unpriv = unpriv
        self.sensitive_attribute = attribute


    def fit(self, X, y):
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
        self.naive_classifier.fit(X_train, y_train)

        # now we are using the validation set to look for a treshold

        mask_priv = (X_valid[:, self.sensitive_attribute] == self.priv)
        X_priv = X_valid[mask_priv, :]
        y_priv = y_valid[mask_priv]
        mask_unpriv = (X_valid[:, self.sensitive_attribute] == self.unpriv)
        X_unpriv = X_valid[mask_unpriv, :]
        y_unpriv = y_valid[mask_unpriv]

        tresholds1 = np.linspace(0.3, 0.7, 50)
        tresholds2 = np.linspace(0.3, 0.7, 50)

        min_diff = 10000
        t_priv, t_unpriv = None, None
        print(self.naive_classifier.predict_proba(X_priv).shape)


        # we look for the pair of tresholds that minimizez 
        # the difference between the demographic parity of the priviledged and unpriviledged groups
        for t1 in tresholds1:
            for t2 in tresholds2:
                pred_priv = np.where(self.naive_classifier.predict_proba(X_priv)[:,1] > t1, 1, 0)
                pred_unpriv = np.where(self.naive_classifier.predict_proba(X_unpriv)[:,1] > t2, 1, 0)

                diff = abs(dp_diff(pred_priv, pred_unpriv))
                if diff < min_diff:
                    min_diff = diff 
                    t_priv = t1
                    t_unpriv = t2

        self.treshold_priv = t_priv
        self.treshold_unpriv = t_unpriv 


    def predict(self, X): 
        # predict using the different tresholds
        predictions = [] 
        for x in X:
            if x[self.sensitive_attribute] == self.priv:
                t = self.treshold_priv
            else:
                t = self.treshold_unpriv 
            
            pred = self.naive_classifier.predict_proba(np.array([x]))[:,1]
            if pred > t:
                predictions.append(1)
            else:
                predictions.append(0)

        return np.array(predictions)









In [65]:
data = pd.read_csv('compas_processed.csv')
y = data['not_recidivist'].to_numpy()
X = data.drop(['not_recidivist'], axis=1).to_numpy()
print(X.shape)
print(y.shape)


(5278, 10)
(5278,)


In [66]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train[0]

array([ 0.4927064 , -0.81385638, -0.14965002, -0.19056396, -0.2472656 ,
       -0.29977561, -0.31491561, -1.15917837,  1.95338   , -0.52957189])

In [68]:
sensitive_attribute_position = 1 
values = set(X_train[:,sensitive_attribute_position])
priv = max(values)
unpriv = min(values)

print(priv)
print(unpriv)


1.2287180226062888
-0.8138563784381182


In [69]:

classifier = HopefullyFairClassifier(priv, unpriv, sensitive_attribute_position)
classifier.fit(X_train, y_train)


(308, 2)


In [70]:
preds = classifier.predict(X_test)
print(preds.shape)
print(y_test.shape)

(1584,)
(1584,)


### When evaluating the model, our purpose is to obtain a fair model, without sacrificing too much accuracy. 
### Compared to the previous naive classifier, overall we obtained similar accuracy and precision, just slightly less compared to the previous three classifiers. However, the recall was visibly higher than any of the previous classifiers. 

In [71]:
def evaluate(name, y_true, y_pred):
    print("For " + name + " classifier, we obtained: ")
    print("Accuracy: ", accuracy_score(y_true, y_pred) )
    print("Precision: ", precision_score(y_true, y_pred) )
    print("Recall: ", recall_score(y_true, y_pred) ) 


evaluate("", y_test, preds)

For  classifier, we obtained: 
Accuracy:  0.6439393939393939
Precision:  0.6246719160104987
Recall:  0.8409893992932862


In [72]:
mask = (X_test[:, 1] == priv)
X_test_white = X_test[mask, :]
y_test_white = y_test[mask]

mask = (X_test[:, 1] == unpriv)
X_test_black = X_test[mask, :]
y_test_black = y_test[mask]



In [73]:
y_pred_black = classifier.predict(X_test_black)
y_pred_white = classifier.predict(X_test_white)

### The demographic parity is respected when the disparate impact is between 0.8 and 1.2. Perfectly fair means that our result is precisely 1. Our classifier is fair with respect to this measure. 

In [74]:
di = disparate_impact(y_pred_white, y_pred_black)
di

0.9744560075685904

In [75]:
evaluate('log reg black', y_test_black, y_pred_black)
evaluate('log reg white', y_test_white, y_pred_white)

For log reg black classifier, we obtained: 
Accuracy:  0.6376811594202898
Precision:  0.5898550724637681
Recall:  0.8586497890295358
For log reg white classifier, we obtained: 
Accuracy:  0.6537216828478964
Precision:  0.6777041942604857
Recall:  0.8186666666666667
